In [1]:
##will soon become a python file

import numpy as np
import gensim 
import nltk

C:\Users\Linda\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def stop_words(sentence,language):
    from nltk import word_tokenize
    from nltk.corpus import stopwords
    stop = set(stopwords.words('english'))
    return([i for i in sentence.lower().split() if i not in stop])
    [i for i in word_tokenize(sentence.lower()) if i not in stop] 


In [3]:
#Load text and treat
def load_care(path): 
    fp = open(path).read()
    aux = fp.split('*')
    text_en = []
    for item in aux:
        stringlist = []
        for letter in item:
            if letter == ',' or letter == '.' or letter == ';' or letter == '(' or letter == ')' or letter == '/' or letter == '"""' or letter == '"\"' or letter == '"':
                letter = ''
            elif letter.isdigit() == True:
                letter = ''
            stringlist.append(letter)
        text_en.append(''.join(stringlist))
    return text_en


In [4]:
retorno = load_care('./teste_en.txt')

In [5]:
src_path = '~/desktop/embeddings/wiki.en.align.vec'
tgt_path = '~/desktop/embeddings/wiki.es.align.vec'
nmax = 100000  
src_model = gensim.models.KeyedVectors.load_word2vec_format(src_path,limit = nmax)
tgt_model = gensim.models.KeyedVectors.load_word2vec_format(tgt_path,limit = nmax)

In [7]:
i =0
for item in retorno:
    if src_model.vocab == True:
        item = stop_words(item,'english')
        print(i)
        embedder(item,'english')
    

In [6]:
def embedder(sentence,language):                                                                     
    first_word = 0 
    for word in sentence:                                 
        if (first_word == 0):
            if(language == 'english'):
                word_emb = src_model[word]
            else:
                word_emb = tgt_model[word]
            first_word = 1
        
        else:
            if(language == 'english'):
                word_emb = word_emb + src_model[word]   
            else:
                word_emb = word_emb + tgt_model[word]
    return word_emb